In [1]:
# Dependencies
import requests
import json
import numpy as np
import pandas as pd

In [2]:
# Build a dictionary of HHS national data.
df1 = pd.DataFrame()
year_list = [2012, 2013, 2014, 2015, 2016, 2017]
for year_entry in year_list:
    url = "https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/{}/national.json?ethnicity=T&medicare_status=A".format(year_entry)
    print(url)
    national_dict = requests.get(url).json()
    df = pd.DataFrame(national_dict)
    df1 = df1.append(df, ignore_index=True)
    print(str(len(df1.index)))

https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/2012/national.json?ethnicity=T&medicare_status=A
10
https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/2013/national.json?ethnicity=T&medicare_status=A
62
https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/2014/national.json?ethnicity=T&medicare_status=A
114
https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/2015/national.json?ethnicity=T&medicare_status=A
166
https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/2016/national.json?ethnicity=T&medicare_status=A
218
https://fluvaccineapi.hhs.gov/api/v2/vaccination_rates/trends/2017/national.json?ethnicity=T&medicare_status=A
243


In [3]:
df1.head()

,count,disparity,ethnicity,fips,medicare_status,name,percentage,short_name,week,week_start,year
0,26043833,0.0,T,0,A,National,0.512392,US,43,01JUN,2012
1,26043833,0.0,T,0,A,National,0.512401,US,44,08JUN,2012
2,26043833,0.0,T,0,A,National,0.512409,US,45,15JUN,2012
3,26043833,0.0,T,0,A,National,0.512415,US,46,22JUN,2012
4,26043833,0.0,T,0,A,National,0.512419,US,47,29JUN,2012


In [4]:
df1.to_json('data/raw_hhs_data.json')

In [5]:
df2 = pd.read_json('data/raw_hhs_data.json')

In [6]:
df2.head()

,count,disparity,ethnicity,fips,medicare_status,name,percentage,short_name,week,week_start,year
0,26043833,0,T,0,A,National,0.512392,US,43,01JUN,2012
1,26043833,0,T,0,A,National,0.512401,US,44,08JUN,2012
10,27278865,0,T,0,A,National,0.000349,US,1,10AUG,2013
100,28830312,0,T,0,A,National,0.501679,US,39,02MAY,2014
101,28830312,0,T,0,A,National,0.501692,US,40,09MAY,2014


In [7]:
df3 = df2[['count', 'percentage', 'year', 'week']]

In [8]:
df4 = df3.rename(columns={"count":"vaccinations", "percentage":"vac_percent"})

In [9]:
df5 = df4.sort_values(by=['year', 'week'])

In [10]:
df5.head()

,vaccinations,vac_percent,year,week
0,26043833,0.512392,2012,43
1,26043833,0.512401,2012,44
2,26043833,0.512409,2012,45
3,26043833,0.512415,2012,46
4,26043833,0.512419,2012,47


In [11]:
df5.head()

,vaccinations,vac_percent,year,week
0,26043833,0.512392,2012,43
1,26043833,0.512401,2012,44
2,26043833,0.512409,2012,45
3,26043833,0.512415,2012,46
4,26043833,0.512419,2012,47


In [23]:
df5['vac_pct_week'] = 0.0
df5['vac_percent'] = df5['vac_percent'] * 100

In [24]:
df5.head()

,vaccinations,vac_percent,year,week,vac_pct_week
0,26043833,51.2392,2012,43,0.0
1,26043833,51.2401,2012,44,0.0
2,26043833,51.2409,2012,45,0.0
3,26043833,51.2415,2012,46,0.0
4,26043833,51.2419,2012,47,0.0


In [25]:
for index_entry in df5.index:
    if df5.loc[index_entry, 'week'] == 1:
        df5.loc[index_entry, 'vac_pct_week'] = df5.loc[index_entry, 'vac_percent']
    elif index_entry > 0:
        i = index_entry - 1
        df5.loc[index_entry, 'vac_pct_week'] = df5.loc[index_entry, 'vac_percent'] - df5.loc[i, 'vac_percent']

In [28]:
df5.head()

,vaccinations,vac_percent,year,week,vac_pct_week
0,26043833,51.2392,2012,43,0.0000
1,26043833,51.2401,2012,44,0.0009
2,26043833,51.2409,2012,45,0.0008
3,26043833,51.2415,2012,46,0.0006
4,26043833,51.2419,2012,47,0.0004


In [29]:
df5.to_json('data/hhs_national_10_years.json')

In [30]:
# Sample code to access the json file.
df6 = pd.read_json('data/hhs_national_10_years.json')

In [31]:
df6.head()

,vac_pct_week,vac_percent,vaccinations,week,year
0,0.000000,51.239200,26043833,43,2012
1,0.000900,51.240100,26043833,44,2012
10,0.034869,0.034869,27278865,1,2013
100,0.001700,50.167900,28830312,39,2014
101,0.001300,50.169200,28830312,40,2014


In [32]:
# Sample code to access the json file.
df7 = df6.sort_values(['year', 'week'])
df7

,vac_pct_week,vac_percent,vaccinations,week,year
0,0.000000,51.239200,26043833,43,2012
1,0.000900,51.240100,26043833,44,2012
2,0.000800,51.240900,26043833,45,2012
3,0.000600,51.241500,26043833,46,2012
4,0.000400,51.241900,26043833,47,2012
5,0.000500,51.242400,26043833,48,2012
6,0.000500,51.242900,26043833,49,2012
7,0.000600,51.243500,26043833,50,2012
8,0.000500,51.244000,26043833,51,2012
9,0.000800,51.244800,26043833,52,2012
